## Experimenting with Graph Theory for Feature Generation

The grid, as defined in the PyPownet environment is really a directed graph with edge weights. We can model it as such, and then use existing graph-theory algorithms to pull out information about the grid that may be beneficial for targeting specific actions given a state.

For example, graph theory can tell us which substations are the most critical to the grid and then we could use this information to either target or avoid them for manipulation.

I created a wrapper class, `ObservationSpaceGraph` that lets us quickly generate graphs from the observation space array, and then we can use the Python package NetworkX to call the graph theory algorithms during agent training.

This example notebook uses the code from the simple, four node grid in the original 101 notebook.

In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from utils.visualize_grid import plot_grid
from matplotlib.font_manager import FontProperties
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Connect Plotly in offline mode. 
init_notebook_mode(connected = True) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [16]:
import os
import pypownet.environment
import pypownet.runner
import networkx as nx

# Initialize the env.
environment = pypownet.environment.RunEnv(parameters_folder=os.path.abspath('../public_data'),
                                          game_level='4_substations',
                                          chronic_looping_mode='natural', start_id=0,
                                          game_over_mode='hard' )

Using custom reward signal CustomRewardSignal of file /home/nadirsidi/Code/L2RPN/public_data/reward_signal.py


In [17]:
action_space = environment.action_space
observation_space = environment.observation_space
game = environment.game

# Create do_nothing action.
action_do_nothing = action_space.get_do_nothing_action()

In [18]:
# Run one step in the environment
obs_array, *_ = environment.step(action_do_nothing)

In [19]:
# Get the Observation object from the array
obs = observation_space.array_to_observation(obs_array)
type(obs)

pypownet.environment.Observation

In [20]:
# Get the __dict__ attribute & pull out the useful nuggets
observation_dict = vars(obs)
# observation_dict

## Example Using ObservationSpaceGraph Wrapper

Create the `obs_space_graph` wrapper by sending it the `observation_space` from environment. Then after each action, we can get the graphs from the observation array returned by `environment.step`

Once we have the graphs as Networkx graph objects, we can use the graph algorithms to experiment with different features such as the substations with the highest degree of centrality in the grid.

In [21]:
from graph import ObservationSpaceGraph

obs_space_graph = ObservationSpaceGraph(observation_space)

graphs = obs_space_graph.get_graph(obs_array)

# This was originally designed for ranking web pages
# The result is a ranking of the substations in the grid taking into account their connectedness & weight (flows)
# We might want to toggle the substations with the highest or lowest values
nx.pagerank(graphs['obs_G'])

# The sum of the fraction of all-pairs shortest paths that passes through node in key
# This is a potential measure of how critical a substation is for connecting the grid
nx.betweenness_centrality(graphs['obs_G'])

# Measure of how central a substation is in the grid
nx.degree_centrality(graphs['obs_G'])

# Measure of how central a substation is as a producer (uses a directed graph)
nx.out_degree_centrality(graphs['obs_DG'])

# Measure of how central a substation is as a consumer (uses a directed graph)
nx.in_degree_centrality(graphs['obs_DG'])


{1.0: 0.0, 2.0: 0.3333333333333333, 3.0: 0.3333333333333333, 4.0: 1.0}